### Modelos de Machine Learning

#### Vamos começar importando novamente a base e fazendo os tratamentos anteriores

In [1]:
# Importando a biblioteca pandas
import pandas as pd

In [2]:
# Visualizando a base de treino
base = pd.read_csv('Churn.csv', sep = (';'))

In [3]:
# Dando nome às colunas
base.columns = ["Id","Score","Estado","Genero","Idade","Patrimonio","Saldo","Produtos","TemCartCredito",
                    "Ativo","Salario","Saiu"]


In [4]:
#Excluimso pelo ID
base.drop_duplicates(subset="Id", keep='first',inplace=True)

In [5]:
# Como masculino é a moda, substituiremos valores nulos por 'Masculino'
base.loc[base.Genero.isnull(),'Genero'] = 'Masculino'

In [6]:
# Descobrindo a mediana para poder substituir posteriormente
import statistics  as sts
mediana = sts.median(base['Salario'])

In [7]:
# Sabendo a mediana, substituiremos valores nulos pela
base.loc[base.Salario.isnull(),'Salario'] = mediana

In [8]:
#Substituindo os valores 'M' por 'Masculino', 'F' e 'Fem' por 'Feminino'
base.loc[base['Genero'] ==  'M', 'Genero'] = 'Masculino'
base.loc[base['Genero'] ==  'F', 'Genero'] = 'Feminino'
base.loc[base['Genero'] ==  'Fem', 'Genero'] = 'Feminino'

In [9]:
#Substituindo valores fora do domínio pela moda ('RS')
base.loc[base['Estado'].isin(['RP','SP','TD']), 'Estado'] = 'RS'      

In [10]:
#Tratando os valores (substituiremos os valores fora do domínio pela mediana)
mediana = sts.median(base['Idade'])
base.loc[(base['Idade'] <  0 )  | ( base['Idade'] >  120), 'Idade'] = mediana

In [11]:
#Por último só ajustaremos a coluna salário. Definiremos outliers em salário dados com mais de 2 desvios padrão.
import statistics  as sts
desv = sts.stdev(base['Salario'])

In [12]:
#Substituiremos esses valores pela mediana
mediana = sts.median(base['Salario'])
base.loc[base['Salario'] >=  2 * desv, 'Salario'] = mediana

### Agora começaremos o teste de 3 dos principais modelos de machine learning: Árvores de classificação, Regressão Logística e KNeighborsClassifier

##### Primeiramente, vamos agora adicionar um novo tratamento das colunas de texto

In [13]:
# Visualizando a base de treino
base

,Id,Score,Estado,Genero,Idade,Patrimonio,Saldo,Produtos,TemCartCredito,Ativo,Salario,Saiu
0,1,619,RS,Feminino,42,2,0,1,1,1,10134888.0,1
1,2,608,SC,Feminino,41,1,8380786,1,0,1,11254258.0,0
2,3,502,RS,Feminino,42,8,1596608,3,1,0,11393157.0,1
3,4,699,RS,Feminino,39,1,0,2,0,0,9382663.0,0
4,5,850,SC,Feminino,43,2,12551082,1,1,1,790841.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
994,996,838,SC,Masculino,43,9,12310588,2,1,0,14576583.0,0
995,997,610,SC,Masculino,29,9,0,3,0,1,8391224.0,0
996,998,811,SC,Masculino,44,3,0,2,0,1,7843973.0,0
997,999,587,SC,Masculino,62,7,12128627,1,0,1,677692.0,0


In [14]:
# Verificando as colunas de texto na base de treino
base.columns[base.dtypes == 'object']

Index(['Estado', 'Genero'], dtype='object')

In [15]:
# Verificando os valores na coluna Estado
base.Estado.value_counts()

Estado
RS    483
SC    258
PR    257
Name: count, dtype: int64

In [16]:
# e na coluna Gênero
base.Genero.value_counts()

Genero
Masculino    535
Feminino     463
Name: count, dtype: int64

#### Para tratar a coluna Genero, podemos criar uma nova coluna chamada "Masculino_Check" que vai receber 1 se o gênero for masculino e 0 se o gênero for feminino

In [17]:
# Usando uma lambda function para fazer esse tratamento
base['MasculinoCheck'] = base.Genero.apply(lambda x: 1 if x == 'Masculino' else 0)

In [18]:
# Verificando os valores
base[['Genero','MasculinoCheck']].value_counts()

Genero     MasculinoCheck
Masculino  1                 535
Feminino   0                 463
Name: count, dtype: int64

#### Agora, para tratar a coluna Estado usando o OneHotEncoder que irá criar uma nova coluna para cada um dos rótulos da coluna original

In [19]:
# Importando o OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

In [20]:
# Criando o encoder
ohe = OneHotEncoder(handle_unknown='ignore',dtype='int32')

In [21]:
# Fazendo o fit com os dados
ohe = ohe.fit(base[['Estado']])

In [22]:
# Fazendo a transformação
ohe.transform(base[['Estado']]).toarray()

array([[0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0]])

In [23]:
# Transformando esse resultado em um DataFrame
ohe_df = pd.DataFrame(ohe.transform(base[['Estado']]).toarray(),columns=ohe.get_feature_names_out())
ohe_df.head(3)

,Estado_PR,Estado_RS,Estado_SC
0,0,1,0
1,0,0,1
2,0,1,0


In [24]:
# Podemos agora adicionar essa coluna na nossa base de treino
base = pd.concat([base,ohe_df],axis=1)

In [25]:

# Verificando os valores
base[['Estado_PR','Estado_RS','Estado_SC']].value_counts()

Estado_PR  Estado_RS  Estado_SC
0.0        1.0        0.0          483
           0.0        1.0          258
1.0        0.0        0.0          257
Name: count, dtype: int64

### Usando essa nova base no modelo

In [26]:
# Podemos então apagar essas duas colunas que já tratamos
base = base.drop(['Estado','Genero'],axis=1)

In [27]:
base

,Id,Score,Idade,Patrimonio,Saldo,Produtos,TemCartCredito,Ativo,Salario,Saiu,MasculinoCheck,Estado_PR,Estado_RS,Estado_SC
0,1.0,619.0,42.0,2.0,0.0,1.0,1.0,1.0,10134888.0,1.0,0.0,0.0,1.0,0.0
1,2.0,608.0,41.0,1.0,8380786.0,1.0,0.0,1.0,11254258.0,0.0,0.0,0.0,0.0,1.0
2,3.0,502.0,42.0,8.0,1596608.0,3.0,1.0,0.0,11393157.0,1.0,0.0,0.0,1.0,0.0
3,4.0,699.0,39.0,1.0,0.0,2.0,0.0,0.0,9382663.0,0.0,0.0,0.0,1.0,0.0
4,5.0,850.0,43.0,2.0,12551082.0,1.0,1.0,1.0,790841.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,997.0,610.0,29.0,9.0,0.0,3.0,0.0,1.0,8391224.0,0.0,1.0,0.0,0.0,1.0
996,998.0,811.0,44.0,3.0,0.0,2.0,0.0,1.0,7843973.0,0.0,1.0,0.0,0.0,1.0
997,999.0,587.0,62.0,7.0,12128627.0,1.0,0.0,1.0,677692.0,0.0,1.0,1.0,0.0,0.0
998,1000.0,811.0,28.0,4.0,16773882.0,2.0,1.0,1.0,990342.0,0.0,0.0,NaN,NaN,NaN


In [28]:
#Verificando possíveis adições de dados faltantes
base.isnull().sum().sort_values(ascending=False)

Id                1
Score             1
Idade             1
Patrimonio        1
Saldo             1
Produtos          1
TemCartCredito    1
Ativo             1
Salario           1
Saiu              1
MasculinoCheck    1
Estado_PR         1
Estado_RS         1
Estado_SC         1
dtype: int64

In [29]:
#Excluindo a linha com valores faltantes
base = base.drop(81)

In [30]:
#Verificando outras linhas com dados faltantes
linhas_com_nulos = base[base.isnull().any(axis=1)]
linhas_com_nulos

,Id,Score,Idade,Patrimonio,Saldo,Produtos,TemCartCredito,Ativo,Salario,Saiu,MasculinoCheck,Estado_PR,Estado_RS,Estado_SC
998,1000.0,811.0,28.0,4.0,16773882.0,2.0,1.0,1.0,990342.0,0.0,0.0,NaN,NaN,NaN


In [31]:
#Substituindo os dados faltantes da linha 998 pela moda (RS)
base.loc[base.Estado_PR.isnull(),'Estado_PR'] = 0.0
base.loc[base.Estado_RS.isnull(),'Estado_RS'] = 1.0
base.loc[base.Estado_SC.isnull(),'Estado_SC'] = 0.0


In [32]:
#Verificando novamente dados faltantes restantes
base.isnull().sum().sort_values(ascending=False)

Id                0
Score             0
Idade             0
Patrimonio        0
Saldo             0
Produtos          0
TemCartCredito    0
Ativo             0
Salario           0
Saiu              0
MasculinoCheck    0
Estado_PR         0
Estado_RS         0
Estado_SC         0
dtype: int64

In [33]:
# Importando o train_test_split
from sklearn.model_selection import train_test_split

In [34]:
# Separando previsores a classe
X = base.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13]]  # Todas as linhas, colunas 0 a 8
y = base.iloc[:, 9]   # Todas as linhas, apenas a coluna 9
y

0      1.0
1      0.0
2      1.0
3      0.0
4      0.0
      ... 
994    0.0
995    0.0
996    0.0
997    0.0
998    0.0
Name: Saiu, Length: 998, dtype: float64

In [35]:
# Separando em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

#### -Para Árvore de Decisão

In [36]:
# Fazendo a importação
from sklearn import tree

In [37]:
# Criando o classificador
clf_ac = tree.DecisionTreeClassifier(random_state=42)

In [38]:
# Fazendo o fit com os dados
clf_ac = clf_ac.fit(X_train,y_train)

In [39]:
# Fazendo a previsão
y_pred_ac = clf_ac.predict(X_val)

#### -Para o KNeighbors

In [40]:
# Importando
from sklearn.neighbors import KNeighborsClassifier

In [41]:
# Criando o classificador
clf_knn = KNeighborsClassifier(n_neighbors=3)

In [42]:
# Fazendo o fit com os dados
clf_knn = clf_knn.fit(X_train,y_train)

In [43]:
# Fazendo a previsão
y_pred_knn = clf_knn.predict(X_val)

#### -Para a Regressão Logística

In [44]:
# Importando
from sklearn.linear_model import LogisticRegression

In [45]:
# Criando o classificador
clf_rl = LogisticRegression(random_state=42,max_iter=1000)

In [46]:
# Fazendo o fit com os dados
clf_rl = clf_rl.fit(X_train,y_train)

In [47]:
# Fazendo a previsão
y_pred_rl = clf_rl.predict(X_val)

### Agora vamos avaliar a acurácia dos modelos

In [240]:
# Importando
from sklearn.metrics import accuracy_score

In [241]:
# Para a árvore
acuracia_ac = accuracy_score(y_val, y_pred_ac)
acuracia_ac

0.8303030303030303

In [242]:
# Para o knn
acuracia_kmn = accuracy_score(y_val, y_pred_knn)
acuracia_kmn

0.7181818181818181

In [243]:
# Para a regressão logística
acuracia_rl = accuracy_score(y_val, y_pred_rl)
acuracia_rl

0.7878787878787878

### Avaliando Matriz de Confusão

In [244]:
# Importando
from sklearn.metrics import confusion_matrix

In [245]:
# Para a árvore
confusion_matrix(y_val, y_pred_ac)

array([[244,  21],
       [ 35,  30]], dtype=int64)

In [246]:
# Para o knn
confusion_matrix(y_val, y_pred_knn)

array([[231,  34],
       [ 59,   6]], dtype=int64)

In [247]:
# Para a regressão logística
confusion_matrix(y_val, y_pred_rl)

array([[257,   8],
       [ 62,   3]], dtype=int64)

### Criando um dataframe para guardar os resultados

In [257]:
# Criando primeiramente o DataFrame
as_df = pd.DataFrame({
     'Modelos': ['arvore','knn','reg. log.'],
     'Acuracia': [acuracia_ac, acuracia_kmn, acuracia_rl]
 })
 
as_df

,Modelos,Acuracia
0,arvore,0.830303
1,knn,0.718182
2,reg. log.,0.787879


In [258]:
# Visualizando
as_df

,Modelos,Acuracia
0,arvore,0.830303
1,knn,0.718182
2,reg. log.,0.787879


#### É notório que o modelo de Árvore de decisão obteve uma acurácia melhor.
#### Agora, iremos fazer as contatações finais.